# Step 1: Import Necessary Libraries
**First, import the necessary libraries.**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


# Step 2: Define the Residual Block

**Define the basic building block of ResNet, the residual block.**

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        # First convolutional layer in the block
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        # Batch normalization after the first convolution
        self.bn1 = nn.BatchNorm2d(out_channels)
        # ReLU activation function
        self.relu = nn.ReLU(inplace=True)
        # Second convolutional layer in the block
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        # Batch normalization after the second convolution
        self.bn2 = nn.BatchNorm2d(out_channels)
        # Downsample layer for matching dimensions if needed
        self.downsample = downsample

    def forward(self, x):
        # Save the input tensor as the residual (shortcut connection)
        residual = x
        # Pass the input through the first convolutional layer
        out1 = self.conv1(x)
        # Apply batch normalization
        out2 = self.bn1(out1)
        # Apply ReLU activation function
        out3 = self.relu(out2)
        # Pass the output through the second convolutional layer
        out4 = self.conv2(out3)
        # Apply batch normalization
        out5 = self.bn2(out4)

        # If downsampling is needed, apply the downsampling layer
        if self.downsample:
            residual = self.downsample(x)

        # Add the residual (shortcut connection) to the output
        out5 += residual
        # Apply ReLU activation function to the final output
        out6 = self.relu(out5)
        return out6


# Step 3: Define the ResNet Model
**Define the ResNet model using the residual blocks.**

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64  # Initial number of input channels for the first layer
        # Initial convolutional layer with 7x7 kernel, stride of 2, and padding of 3
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Batch normalization for the first convolutional layer
        self.bn1 = nn.BatchNorm2d(64)
        # ReLU activation function
        self.relu = nn.ReLU(inplace=True)  # inplace=True for memory optimization
        # Max pooling layer with 3x3 kernel, stride of 2, and padding of 1
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # Creating the first block of layers (layer1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        # Creating the second block of layers (layer2) with a stride of 2
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        # Creating the third block of layers (layer3) with a stride of 2
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        # Creating the fourth block of layers (layer4) with a stride of 2
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        # Adaptive average pooling layer that outputs 1x1 spatial dimensions
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # Fully connected layer for classification
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        # If the stride is not 1 or the input channels don't match the output channels after expansion, perform downsampling
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion)
            )

        layers = []
        # Add the first block, which may include downsampling
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        # Add the remaining blocks
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x1 = self.conv1(x)  # Apply the initial convolutional layer
        x2 = self.bn1(x1)  # Apply batch normalization
        x3 = self.relu(x2)  # Apply ReLU activation
        x4 = self.maxpool(x3)  # Apply max pooling
        x5 = self.layer1(x4)  # Pass through the first block of layers
        x6 = self.layer2(x5)  # Pass through the second block of layers
        x7 = self.layer3(x6)  # Pass through the third block of layers
        x8 = self.layer4(x7)  # Pass through the fourth block of layers
        x9 = self.avgpool(x8)  # Apply adaptive average pooling
        x10 = torch.flatten(x9, 1)  # Flatten the output to feed into the fully connected layer
        x11 = self.fc(x10)  # Apply the fully connected layer
        return x11  # Return the final output


# Creating a ResNet Model
**You can create specific versions of ResNet by specifying the number of blocks for each layer. For example, ResNet-18 and ResNet-34 have different numbers of blocks in each layer:**

In [ ]:
def resnet18():
    return ResNet(ResidualBlock, [2, 2, 2, 2])

def resnet34():
    return ResNet(ResidualBlock, [3, 4, 6, 3])


# Example Usage
**Here's how you can create and use a ResNet-18 model:**

In [ ]:
model = resnet18()
print(model)

# Define a dummy input tensor with the shape (batch_size, channels, height, width)
dummy_input = torch.randn(1, 3, 224, 224)

# Forward pass through the model
output = model(dummy_input)
print(output.shape)


# Step 4: Prepare the Data
**Load and preprocess the dataset.**

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Step 5: Train the Model
**Train the ResNet model.**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = resnet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Save the model
torch.save(model.state_dict(), 'ModelResNet18.pth')


# Step 6: Evaluate the Model
**Evaluate the trained model on the test dataset.**

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')
